In [1]:
cd /home/ubuntu/fedatk_unl_tj/

/home/ubuntu/fedatk_unl_tj


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from utils.util_notebooks import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

In [3]:
setting, num_user = "FedAvg", 40
num_models = 2

try: # Skip loading if already loaded
    aggregator
except:
    aggregator, clients, args_ = set_args(setting, num_user)

# Load models for FAT and FedAvg
save_path_FAT = 'weights/cifar10/230922_baseline_train/FAT/'
save_path_FedAvg = 'weights/cifar10/230922_baseline_train/fedavg/'

model_FAT = copy.deepcopy(import_model_weights(num_user, setting, save_path_FAT, aggregator, args_)[0])
model_Fedavg = import_model_weights(num_user, setting, save_path_FedAvg, aggregator, args_)[0]

# dataloader_agg = get_aggregate_dataloader(clients)
logs_adv = generate_logs_adv(num_models)


==> Clients initialization..
===> Building data iterators..


100%|██████████| 80/80 [00:00<00:00, 165.86it/s]


===> Initializing clients..


100%|██████████| 80/80 [00:15<00:00,  5.28it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.299 | Train Acc: 10.548% |Test Loss: 2.297 | Test Acc: 10.511% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [4]:
# Perform transfer attack from one client to another and record stats

models_test = [model_Fedavg, model_FAT]

# Run Measurements for both targetted and untargeted analysis
victim_idxs = range(len(models_test))
custom_batch_size = 500
eps = 3

dataloader = load_client_data(clients = clients, c_id = 0, mode = 'all') # or test/train
cross_attack(logs_adv, victim_idxs, dataloader, models_test, custom_batch_size, eps)

	 Adv idx: 0


NameError: name 'eps' is not defined

In [ ]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
            'adv_similarities_untarget','adv_target','adv_miss']

get_metric_list("orig_acc_transfers", logs_adv, victim_idxs)

array([[0.85, 0.82],
       [0.85, 0.78]])

In [ ]:
logs_adv[0]["adv_miss"]

{0: tensor(0.0080, device='cuda:0'), 1: tensor(0.1900, device='cuda:0')}